In [1]:
import os

# 参数设置
models = ['vicuna', 'llama2']
select_search_values = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
cosdecays=[True,False]
betas=[1,0]
similarities=['mean','llm']
# Shell脚本模板
script_template = """#!/bin/bash
export WANDB_MODE=disabled
# Optionally set the cache for transformers
# export TRANSFORMERS_CACHE='YOUR_PATH/huggingface'
export n=25
export model='{model}'
export device=0
export beta=1
export cosinedecay={cosdecay}
export search={search}
export select={select}
export folder={folder}
export similarity={similarity}
# Create results folder if it doesn't exist
if [ ! -d "../results" ]; then
    mkdir "../results"
    echo "Folder '../results' created."
else
    echo "Folder '../results' already exists."
fi

export round=1

mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/launch_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results

nohup python -u ../main.py \\
    --config="../configs/transfer_{model}.py" \\
    --config.attack=gcg \\
    --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
    --config.result_prefix="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}" \\
    --config.progressive_goals=False \\
    --config.stop_on_success=False \\
    --config.num_train_models=1 \\
    --config.allow_non_ascii=False \\
    --config.n_train_data=$n \\
    --config.n_steps=500 \\
    --config.test_steps=50 \\
    --config.batch_size=512 \\
    --config.searchalpha={search} \\
    --config.selectalpha={select} \\
    --config.beta=1 \\
    --config.cosine_decay={cosdecay} \\
    --config.similarity={similarity} \\
    --config.devices[0]="cuda" > $HOME/llm-attacks/experiments/{folder}/round${{round}}/launch_scripts/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}.log 2>&1 &
pid=$!
wait $pid

export round=2

mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/launch_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results

nohup python -u ../main.py \\
    --config="../configs/transfer_{model}.py" \\
    --config.attack=gcg \\
    --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
    --config.result_prefix="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}" \\
    --config.progressive_goals=False \\
    --config.stop_on_success=False \\
    --config.num_train_models=1 \\
    --config.allow_non_ascii=False \\
    --config.n_train_data=$n \\
    --config.n_steps=500 \\
    --config.test_steps=50 \\
    --config.batch_size=512 \\
    --config.searchalpha={search} \\
    --config.selectalpha={select} \\
    --config.beta=1 \\
    --config.cosine_decay={cosdecay} \\
    --config.similarity={similarity} \\
    --config.devices[0]="cuda" > $HOME/llm-attacks/experiments/{folder}/round${{round}}/launch_scripts/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}.log 2>&1 &
pid=$!
wait $pid

export round=3

mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/launch_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results

nohup python -u ../main.py \\
    --config="../configs/transfer_{model}.py" \\
    --config.attack=gcg \\
    --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
    --config.result_prefix="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}" \\
    --config.progressive_goals=False \\
    --config.stop_on_success=False \\
    --config.num_train_models=1 \\
    --config.allow_non_ascii=False \\
    --config.n_train_data=$n \\
    --config.n_steps=500 \\
    --config.test_steps=50 \\
    --config.batch_size=512 \\
    --config.searchalpha={search} \\
    --config.selectalpha={select} \\
    --config.beta=1 \\
    --config.cosine_decay={cosdecay} \\
    --config.similarity={similarity} \\
    --config.devices[0]="cuda" > $HOME/llm-attacks/experiments/{folder}/round${{round}}/launch_scripts/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}.log 2>&1 &
pid=$!
wait $pid
"""

# 生成脚本


for model in models:
    for beta in betas:
        for cosdecay in cosdecays:
            for select_search in select_search_values:
                for similarity in similarities:
                    if cosdecay==True:
                        output_dir='./generated_scripts/'+model+'/'+str(beta)+'/True'+'/'+similarity
                    else:
                        output_dir='./generated_scripts/'+model+'/'+str(beta)+'/False'+'/'+similarity
                    os.makedirs(output_dir, exist_ok=True)
                    folder=''
                    folder+=model
                    if cosdecay==True:
                        folder += '_with'
                    else :
                        folder += '_without'
                    folder+='_cosdecay_and'
                    if similarity=='mean':
                        folder+='_mean_sentemb'
                    elif similarity=='llm':
                        folder+='_lasthiddenstate'
                    script_content = script_template.format(
                        model=model,
                        select=select_search,
                        search=select_search,
                        cosdecay=cosdecay,
                        beta=beta,
                        folder=folder,
                        similarity=similarity
                    )
                    script_filename = f'run_gcg_multiple_{model}_search_{select_search}_beta_{beta}_select_{select_search}_cosinedecay_{cosdecay}_similarity_{similarity}.sh'
                    script_path = os.path.join(output_dir, script_filename)
                    
                    with open(script_path, 'w') as script_file:
                        script_file.write(script_content)
                    # print(f'Generated script: {script_path}')


In [ ]:
import os

# 参数设置
models = ['vicuna', 'llama2']
beta=[1,0]
select_search_values = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
cosdecays=[True,False]
similarities=['mean','llm']
# SLURM脚本模板

slurm_script_template = """#!/bin/bash
#SBATCH --job-name gcg{search}{model}          
#SBATCH --partition critical
#SBATCH -n 2
#SBATCH --nodes 1
#SBATCH --ntasks 1                  
#SBATCH --cpus-per-task=2
#SBATCH --mem-per-cpu=16GB
#SBATCH --gres gpu:NVIDIAA40:1
#SBATCH --time 5-00:00:00
#SBATCH --output log.log
#SBATCH --mail-user 17755923416@163.com
#SBATCH --mail-type ALL
#SBATCH -A wangwj1-critical

srun bash run_gcg_multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_similarity_{similarity}.sh
"""


for model in models:
    for beta in betas:
        for cosdecay in cosdecays:
            for select_search in select_search_values:
                for similarity in similarities:
                    if cosdecay==True:
                        output_dir='./generated_scripts/'+model+'/'+str(beta)+'/True'+'/'+similarity
                    else:
                        output_dir='./generated_scripts/'+model+'/'+str(beta)+'/False'+'/'+similarity
                    os.makedirs(output_dir, exist_ok=True)
                    slurm_content = slurm_script_template.format(
                        model=model,
                        select=select_search,
                        search=select_search,
                        beta=beta,
                        cosdecay=cosdecay,
                        similarity=similarity,
                    )
                    
                    slurm_filename = f'run_gcg_multiple_{model}_search_{select_search}_beta_{beta}_select_{select_search}_cosinedecay_{cosdecay}_similarity_{similarity}.slurm'
                    slurm_path = os.path.join(output_dir, slurm_filename)
                    
                    with open(slurm_path, 'w') as slurm_file:
                        slurm_file.write(slurm_content)
                    
                    # print(f'Generated SLURM script: {slurm_path}')


In [1]:
import os

# 参数设置
models = ['vicuna']
select_search_values = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
beta=1
cosdecays=[True]
similarities=['mean',]
# 打印 sbatch 命令模板
sbatch_command_template = "sbatch run_gcg_multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_similarity_{similarity}.slurm"

# 生成 sbatch 命令
for model in models:
    for select_search in select_search_values:
        for similarity in similarities:
            for cosdecay in cosdecays:
                sbatch_command = sbatch_command_template.format(
                    model=model,
                    select=select_search,
                    search=select_search,
                    beta=beta,
                    cosdecay=cosdecay,
                    similarity=similarity,
                )
                print(sbatch_command)


sbatch run_gcg_multiple_vicuna_search_0_beta_1_select_0_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_5_beta_1_select_5_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_10_beta_1_select_10_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_15_beta_1_select_15_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_20_beta_1_select_20_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_25_beta_1_select_25_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_30_beta_1_select_30_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_35_beta_1_select_35_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_40_beta_1_select_40_cosinedecay_True_similarity_mean.slurm
sbatch run_gcg_multiple_vicuna_search_45_beta_1_select_45_cosinedecay_True_similarity_mean.slurm


In [6]:
import os

# 参数设置
models = ['vicuna', 'llama2']
select_search_values = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
cosdecays=[True,False]
betas=[1,0]
similarities=['mean','llm']
rounds=[1,2,3]
# Shell脚本模板
script_template = """#!/bin/bash
export WANDB_MODE=disabled
# Optionally set the cache for transformers
# export TRANSFORMERS_CACHE='YOUR_PATH/huggingface'
export n=25
export model='{model}'
export device=0
export beta=1
export cosinedecay={cosdecay}
export search={search}
export select={select}
export folder={folder}
export similarity={similarity}
export round={round}
# Create results folder if it doesn't exist
if [ ! -d "../results" ]; then
    mkdir "../results"
    echo "Folder '../results' created."
else
    echo "Folder '../results' already exists."
fi

mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/launch_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results

nohup python -u ../main.py \\
    --config="../configs/transfer_{model}.py" \\
    --config.attack=gcg \\
    --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
    --config.result_prefix="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}" \\
    --config.progressive_goals=False \\
    --config.stop_on_success=False \\
    --config.num_train_models=1 \\
    --config.allow_non_ascii=False \\
    --config.n_train_data=$n \\
    --config.n_steps=500 \\
    --config.test_steps=50 \\
    --config.batch_size=512 \\
    --config.searchalpha={search} \\
    --config.selectalpha={select} \\
    --config.beta=1 \\
    --config.cosine_decay={cosdecay} \\
    --config.similarity={similarity} \\
    --config.devices[0]="cuda" > $HOME/llm-attacks/experiments/{folder}/round${{round}}/launch_scripts/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}.log 2>&1 &
pid=$!
wait $pid
"""

# 生成脚本


for model in models:
    for beta in betas:
        for cosdecay in cosdecays:
            for round in rounds:
                for select_search in select_search_values:
                    for similarity in similarities:
                        if cosdecay==True:
                            output_dir='./generated_scripts/'+model+'/'+str(beta)+'/True'+'/'+similarity
                        else:
                            output_dir='./generated_scripts/'+model+'/'+str(beta)+'/False'+'/'+similarity
                        os.makedirs(output_dir, exist_ok=True)
                        folder=model
                        if cosdecay==True:
                            folder += '_with'
                        else :
                            folder += '_without'
                        folder+='_cosdecay_and'
                        if similarity=='mean':
                            folder+='_mean_sentemb'
                        elif similarity=='llm':
                            folder+='_lasthiddenstate'
                        script_content = script_template.format(
                            model=model,
                            select=select_search,
                            search=select_search,
                            cosdecay=cosdecay,
                            beta=beta,
                            folder=folder,
                            similarity=similarity,
                            round=round
                        )
                        script_filename = f'run_gcg_multiple_{model}_search_{select_search}_beta_{beta}_select_{select_search}_cosinedecay_{cosdecay}_similarity_{similarity}_round_{round}.sh'
                        script_path = os.path.join(output_dir, script_filename)
                        
                        with open(script_path, 'w') as script_file:
                            script_file.write(script_content)
                        # print(f'Generated script: {script_path}')


In [3]:
import os

# 参数设置
models = ['vicuna', 'llama2']
select_search_values = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
cosdecays=[True,False]
betas=[1,0]
similarities=['mean','llm']
rounds=[4,5,6]
# Shell脚本模板
script_template = """#!/bin/bash
export WANDB_MODE=disabled
# Optionally set the cache for transformers
# export TRANSFORMERS_CACHE='YOUR_PATH/huggingface'
export n=25
export model='{model}'
export device=0
export beta=1
export cosinedecay={cosdecay}
export search={search}
export select={select}
export folder={folder}
export similarity={similarity}
export round={round}
# Create results folder if it doesn't exist
if [ ! -d "../results" ]; then
    mkdir "../results"
    echo "Folder '../results' created."
else
    echo "Folder '../results' already exists."
fi

mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/launch_scripts
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results

nohup python -u ../main.py \\
    --config="../configs/transfer_{model}.py" \\
    --config.attack=gcg \\
    --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
    --config.result_prefix="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}" \\
    --config.progressive_goals=False \\
    --config.stop_on_success=False \\
    --config.num_train_models=1 \\
    --config.allow_non_ascii=False \\
    --config.n_train_data=$n \\
    --config.n_steps=500 \\
    --config.test_steps=50 \\
    --config.batch_size=512 \\
    --config.searchalpha={search} \\
    --config.selectalpha={select} \\
    --config.beta=1 \\
    --config.cosine_decay={cosdecay} \\
    --config.similarity={similarity} \\
    --config.devices[0]="cuda" > $HOME/llm-attacks/experiments/{folder}/round${{round}}/launch_scripts/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}.log 2>&1 &
pid=$!
wait $pid



export eval_method='harmbench'
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval/${{eval_method}}

nohup python -u ../evaluate_{model}.py \\
    --config="../configs/transfer_{model}.py"\\
    --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
    --config.logfile="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}.json"\\
    --config.n_train_data=$n\\
    --config.n_test_data=100\\
    --config.eval_method=${{eval_method}} \\
    --config.devices[0]="cuda:${{device}}" > $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}/eval_${{file}}_eval_method_${{eval_method}}.log 2>&1 &
pid=$!
wait $pid

export eval_method='kw_matching'
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval/${{eval_method}}

nohup python -u ../evaluate_{model}.py \\
    --config="../configs/transfer_{model}.py"\\
    --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
    --config.logfile="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}.json"\\
    --config.n_train_data=$n\\
    --config.n_test_data=100\\
    --config.eval_method=${{eval_method}} \\
    --config.devices[0]="cuda:${{device}}" > $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}/eval_${{file}}_eval_method_${{eval_method}}.log 2>&1 &
pid=$!
wait $pid

eval "$(conda shell.bash hook)"
conda activate gcgeval
export eval_method='llamaguard'
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}
mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval/${{eval_method}}

nohup python -u ../evaluate_{model}.py \\
    --config="../configs/transfer_{model}.py"\\
    --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
    --config.logfile="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_round_${{round}}.json"\\
    --config.n_train_data=$n\\
    --config.n_test_data=100\\
    --config.eval_method=${{eval_method}} \\
    --config.devices[0]="cuda:${{device}}" > $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}/eval_${{file}}_eval_method_${{eval_method}}.log 2>&1 &
pid=$!
wait $pid

"""

# 生成脚本


for model in models:
    for beta in betas:
        for cosdecay in cosdecays:
            for round in rounds:
                for select_search in select_search_values:
                    for similarity in similarities:
                        if cosdecay==True:
                            output_dir='./generated_scripts/'+model+'/'+str(beta)+'/True'+'/'+similarity
                        else:
                            output_dir='./generated_scripts/'+model+'/'+str(beta)+'/False'+'/'+similarity
                        os.makedirs(output_dir, exist_ok=True)
                        folder=model
                        if cosdecay==True:
                            folder += '_with'
                        else :
                            folder += '_without'
                        folder+='_cosdecay_and'
                        if similarity=='mean':
                            folder+='_mean_sentemb'
                        elif similarity=='llm':
                            folder+='_lasthiddenstate'
                        script_content = script_template.format(
                            model=model,
                            select=select_search,
                            search=select_search,
                            cosdecay=cosdecay,
                            beta=beta,
                            folder=folder,
                            similarity=similarity,
                            round=round
                        )
                        script_filename = f'run_gcg_multiple_{model}_search_{select_search}_beta_{beta}_select_{select_search}_cosinedecay_{cosdecay}_similarity_{similarity}_round_{round}.sh'
                        script_path = os.path.join(output_dir, script_filename)
                        
                        with open(script_path, 'w') as script_file:
                            script_file.write(script_content)
                        # print(f'Generated script: {script_path}')

In [4]:
import os

# 参数设置
models = ['vicuna', 'llama2']
beta=[1,0]
rounds=[4,5,6]
select_search_values = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
cosdecays=[True,False]
similarities=['mean','llm']
# SLURM脚本模板

slurm_script_template = """#!/bin/bash
#SBATCH --job-name gcg{search}{model}          
#SBATCH --partition critical
#SBATCH -n 2
#SBATCH --nodes 1
#SBATCH --ntasks 1                  
#SBATCH --cpus-per-task=2
#SBATCH --mem-per-cpu=16GB
#SBATCH --gres gpu:NVIDIAA40:1
#SBATCH --time 5-00:00:00
#SBATCH --output log.log
#SBATCH --mail-user 17755923416@163.com
#SBATCH --mail-type ALL
#SBATCH -A wangwj1-critical

srun bash run_gcg_multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_similarity_{similarity}_round_{round}.sh
"""


for model in models:
    for beta in betas:
        for cosdecay in cosdecays:
            for round in rounds:
                for select_search in select_search_values:
                    for similarity in similarities:
                        if cosdecay==True:
                            output_dir='./generated_scripts/'+model+'/'+str(beta)+'/True'+'/'+similarity
                        else:
                            output_dir='./generated_scripts/'+model+'/'+str(beta)+'/False'+'/'+similarity
                        os.makedirs(output_dir, exist_ok=True)
                        slurm_content = slurm_script_template.format(
                            model=model,
                            select=select_search,
                            search=select_search,
                            beta=beta,
                            cosdecay=cosdecay,
                            similarity=similarity,
                            round=round
                        )
                        
                        slurm_filename = f'run_gcg_multiple_{model}_search_{select_search}_beta_{beta}_select_{select_search}_cosinedecay_{cosdecay}_similarity_{similarity}_round_{round}.slurm'
                        slurm_path = os.path.join(output_dir, slurm_filename)
                        
                        with open(slurm_path, 'w') as slurm_file:
                            slurm_file.write(slurm_content)
                        
                        # print(f'Generated SLURM script: {slurm_path}')


In [5]:
import os

# 参数设置
models = ['vicuna']
select_search_values = [5, 10, 15, 20, 25, 30, 35, 40, 45]
beta=1
cosdecays=[True]
similarities=['mean']
rounds=[2]
# 打印 sbatch 命令模板
sbatch_command_template = "sbatch run_gcg_multiple_{model}_search_{search}_beta_{beta}_select_{select}_cosinedecay_{cosdecay}_similarity_{similarity}_round_{round}.slurm"

# 生成 sbatch 命令
for model in models:
    for select_search in select_search_values:
        for similarity in similarities:
            for cosdecay in cosdecays:
                for round in rounds:
                    sbatch_command = sbatch_command_template.format(
                        model=model,
                        select=select_search,
                        search=select_search,
                        beta=beta,
                        cosdecay=cosdecay,
                        similarity=similarity,
                        round=round
                    )
                    
                    print(sbatch_command)

sbatch run_gcg_multiple_vicuna_search_5_beta_1_select_5_cosinedecay_True_similarity_mean_round_2.slurm
sbatch run_gcg_multiple_vicuna_search_10_beta_1_select_10_cosinedecay_True_similarity_mean_round_2.slurm
sbatch run_gcg_multiple_vicuna_search_15_beta_1_select_15_cosinedecay_True_similarity_mean_round_2.slurm
sbatch run_gcg_multiple_vicuna_search_20_beta_1_select_20_cosinedecay_True_similarity_mean_round_2.slurm
sbatch run_gcg_multiple_vicuna_search_25_beta_1_select_25_cosinedecay_True_similarity_mean_round_2.slurm
sbatch run_gcg_multiple_vicuna_search_30_beta_1_select_30_cosinedecay_True_similarity_mean_round_2.slurm
sbatch run_gcg_multiple_vicuna_search_35_beta_1_select_35_cosinedecay_True_similarity_mean_round_2.slurm
sbatch run_gcg_multiple_vicuna_search_40_beta_1_select_40_cosinedecay_True_similarity_mean_round_2.slurm
sbatch run_gcg_multiple_vicuna_search_45_beta_1_select_45_cosinedecay_True_similarity_mean_round_2.slurm


In [6]:
# import os

# # 参数设置
# models = ['vicuna', 'llama2']
# eval_methods = ['harmbench', 'llamaguard', 'kw_matching']
# rounds = [1, 2, 3]
# folder='_with_cosdecay_and_mean_sentemb'
# # Shell脚本模板
# script_template = """#!/bin/bash
# export model='{model}'
# export n=25
# export eval_method='{eval_method}'
# export device=0
# export folder='{folder}'
# demofun(){{
#     mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}
#     mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval/${{eval_method}}
#     for file in `ls $1`
#     do
#         echo "file: ${{file}}"
#         echo `pwd`
#         nohup python -u ../evaluate_{model}.py \\
#             --config="../configs/transfer_{model}.py"\\
#             --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
#             --config.logfile="$1/${{file}}"\\
#             --config.n_train_data=$n\\
#             --config.n_test_data=100\\
#             --config.eval_method=${{eval_method}} \\
#             --config.devices[0]="cuda:${{device}}" > $HOME/llm-attacks/experiments/${{folder}}/round{round}/eval_scripts/${{eval_method}}/eval_${{file}}_eval_method_{eval_method}.log 2>&1 &
#         pid=$!
#         wait $pid
#     done
# }}

# export round={round}
# path="$HOME/llm-attacks/experiments/${{folder}}/round{round}/results/"
# demofun $path
# """


# for model in models:
#     for eval_method in eval_methods:
#         for round_value in rounds:
#             output_dir='./generated_eval_scripts/'+model
#             os.makedirs(output_dir, exist_ok=True)
#             script_content = script_template.format(
#                 model=model,
#                 eval_method=eval_method,
#                 round=round_value,
#                 folder=model+folder
#             )
            
#             script_filename = f'batch_run_eval_{eval_method}_round{round_value}_model_{model}.sh'
#             script_path = os.path.join(output_dir, script_filename)
            
#             with open(script_path, 'w') as script_file:
#                 script_file.write(script_content)
            
#             print(f'Generated script: {script_path}')

import os

# 参数设置
models = ['vicuna', 'llama2']
eval_methods = ['harmbench', 'llamaguard', 'kw_matching']
rounds = [1, 2, 3]
folders=['_with_cosdecay_and_lasthiddenstate','_with_cosdecay_and_mean_sentemb','_without_cosdecay_and_mean_sentemb','_without_cosdecay_and_lasthiddenstate']
# Shell脚本模板
script_template = """#!/bin/bash
export model='{model}'
export n=25
export eval_method='{eval_method}'
export device=0
export folder='{folder}'
demofun(){{
    mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}
    mkdir -p $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval/${{eval_method}}
    for file in `ls $1`
    do
        echo "file: ${{file}}"
        echo `pwd`
        nohup python -u ../evaluate_{model}.py \\
            --config="../configs/transfer_{model}.py"\\
            --config.train_data="../../data/advbench/harmful_behaviors.csv" \\
            --config.logfile="$1/${{file}}"\\
            --config.n_train_data=$n\\
            --config.n_test_data=100\\
            --config.eval_method=${{eval_method}} \\
            --config.devices[0]="cuda:${{device}}" > $HOME/llm-attacks/experiments/${{folder}}/round${{round}}/eval_scripts/${{eval_method}}/eval_${{file}}_eval_method_{eval_method}.log 2>&1 &
        pid=$!
        wait $pid
    done
}}

export round=1
path="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/"
demofun $path

export round=2
path="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/"
demofun $path

export round=3
path="$HOME/llm-attacks/experiments/${{folder}}/round${{round}}/results/"
demofun $path
"""

for folder in folders:
    for model in models:
        for eval_method in eval_methods:
            for round_value in rounds:
                output_dir='./generated_eval_scripts/'+model
                os.makedirs(output_dir, exist_ok=True)
                script_content = script_template.format(
                    model=model,
                    eval_method=eval_method,
                    folder=model+folder
                )
                
                script_filename = f'batch_run_eval_{eval_method}_model_{model}_folder{folder}.sh'
                script_path = os.path.join(output_dir, script_filename)
                
                with open(script_path, 'w') as script_file:
                    script_file.write(script_content)
                
                print(f'Generated script: {script_path}')

Generated script: ./generated_eval_scripts/vicuna/batch_run_eval_harmbench_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.sh
Generated script: ./generated_eval_scripts/vicuna/batch_run_eval_harmbench_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.sh
Generated script: ./generated_eval_scripts/vicuna/batch_run_eval_harmbench_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.sh
Generated script: ./generated_eval_scripts/vicuna/batch_run_eval_llamaguard_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.sh
Generated script: ./generated_eval_scripts/vicuna/batch_run_eval_llamaguard_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.sh
Generated script: ./generated_eval_scripts/vicuna/batch_run_eval_llamaguard_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.sh
Generated script: ./generated_eval_scripts/vicuna/batch_run_eval_kw_matching_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.sh
Generated script: ./generated_eval_scripts/vicuna/batch_run_eval_kw_mat

In [7]:
# import os

# # 参数设置
# eval_methods = ['harmbench', 'llamaguard', 'kw_matching']
# rounds = [1, 2, 3]
# models = ['vicuna', 'llama2']
# # SLURM脚本模板
# slurm_script_template = """#!/bin/bash
# #SBATCH --job-name gcgeval
# #SBATCH --partition critical
# #SBATCH -n 2
# #SBATCH --nodes 1
# #SBATCH --ntasks 1
# #SBATCH --cpus-per-task=2
# #SBATCH --mem-per-cpu=16GB
# #SBATCH --gres gpu:NVIDIAA40:1
# #SBATCH --time 5-00:00:00
# #SBATCH --mail-user 17755923416@163.com
# #SBATCH --mail-type ALL
# #SBATCH --output log.log
# #SBATCH -A wangwj1-critical

# srun bash batch_run_eval_{eval_method}_round{round}_model_{model}.sh
# """

# # 生成SLURM脚本
# output_dir = 'generated_eval_scripts'


# for eval_method in eval_methods:
#     for round_value in rounds:
#         for model in models:

#             slurm_content = slurm_script_template.format(
#                 eval_method=eval_method,
#                 round=round_value,
#                 model=model
#             )
#             output_dir='./generated_eval_scripts/'+model

#             slurm_filename = f'eval_{eval_method}_round{round_value}_model_{model}.slurm'
#             slurm_path = os.path.join(output_dir, slurm_filename)
            
#             with open(slurm_path, 'w') as slurm_file:
#                 slurm_file.write(slurm_content)
            
#             print(f'Generated SLURM script: {slurm_path}')

import os

# 参数设置
eval_methods = ['harmbench', 'llamaguard', 'kw_matching']
models = ['vicuna', 'llama2']
similarities=['mean','llm']
folders=['_with_cosdecay_and_lasthiddenstate','_with_cosdecay_and_mean_sentemb','_without_cosdecay_and_mean_sentemb','_without_cosdecay_and_lasthiddenstate']
# SLURM脚本模板
slurm_script_template = """#!/bin/bash
#SBATCH --job-name gcgeval_{eval_method}_model_{model}_folder_{folder}
#SBATCH --partition critical
#SBATCH -n 2
#SBATCH --nodes 1
#SBATCH --ntasks 1
#SBATCH --cpus-per-task=2
#SBATCH --mem-per-cpu=16GB
#SBATCH --gres gpu:NVIDIAA40:1
#SBATCH --time 5-00:00:00
#SBATCH --mail-user 794205443@qq.com
#SBATCH --mail-type ALL
#SBATCH --output log.log
#SBATCH -A wangwj1-critical

srun bash batch_run_eval_{eval_method}_model_{model}_folder{folder}_.sh

"""

# 生成SLURM脚本
output_dir = 'generated_eval_scripts'

for folder in folders:
    for eval_method in eval_methods:
        for model in models:

            slurm_content = slurm_script_template.format(
                eval_method=eval_method,
                model=model,
                folder=folder
            )
            output_dir='./generated_eval_scripts/'+model

            slurm_filename = f'eval_{eval_method}_model_{model}_folder{folder}.slurm'
            slurm_path = os.path.join(output_dir, slurm_filename)
            
            with open(slurm_path, 'w') as slurm_file:
                slurm_file.write(slurm_content)
            
            print(f'Generated SLURM script: {slurm_path}')


Generated SLURM script: ./generated_eval_scripts/vicuna/eval_harmbench_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.slurm
Generated SLURM script: ./generated_eval_scripts/llama2/eval_harmbench_model_llama2_folder_with_cosdecay_and_lasthiddenstate.slurm
Generated SLURM script: ./generated_eval_scripts/vicuna/eval_llamaguard_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.slurm
Generated SLURM script: ./generated_eval_scripts/llama2/eval_llamaguard_model_llama2_folder_with_cosdecay_and_lasthiddenstate.slurm
Generated SLURM script: ./generated_eval_scripts/vicuna/eval_kw_matching_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.slurm
Generated SLURM script: ./generated_eval_scripts/llama2/eval_kw_matching_model_llama2_folder_with_cosdecay_and_lasthiddenstate.slurm
Generated SLURM script: ./generated_eval_scripts/vicuna/eval_harmbench_model_vicuna_folder_with_cosdecay_and_mean_sentemb.slurm
Generated SLURM script: ./generated_eval_scripts/llama2/eval_harmbench_model_lla

In [8]:
# import os

# # 参数设置
# models = ['vicuna', 'llama2']
# rounds = [1, 2, 3]
# eval_methods = ['harmbench', 'llamaguard', 'kw_matching']

# # 打印 sbatch 命令模板
# sbatch_command_template = """sbatch eval_{eval_method}_round{round}_model_{model}.slurm"""

# # 生成 sbatch 命令
# for model in models:
#     for round_value in rounds:
#         for eval_method in eval_methods:
#             sbatch_command = sbatch_command_template.format(
#                 model=model,
#                 round=round_value,
#                 eval_method=eval_method
#             )
            
#             print(sbatch_command)
import os

# 参数设置
models = ['vicuna', 'llama2']
folders=['_with_cosdecay_and_lasthiddenstate','_with_cosdecay_and_mean_sentemb','_without_cosdecay_and_mean_sentemb','_without_cosdecay_and_lasthiddenstate']
eval_methods = ['harmbench', 'llamaguard', 'kw_matching']

# 打印 sbatch 命令模板
sbatch_command_template = """sbatch eval_{eval_method}_model_{model}_folder{folder}.slurm"""

# 生成 sbatch 命令
for model in models:
    for folder in folders:
        for eval_method in eval_methods:
            sbatch_command = sbatch_command_template.format(
                model=model,
                eval_method=eval_method,
                folder=folder
            )
            
            print(sbatch_command)


sbatch eval_harmbench_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.slurm
sbatch eval_llamaguard_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.slurm
sbatch eval_kw_matching_model_vicuna_folder_with_cosdecay_and_lasthiddenstate.slurm
sbatch eval_harmbench_model_vicuna_folder_with_cosdecay_and_mean_sentemb.slurm
sbatch eval_llamaguard_model_vicuna_folder_with_cosdecay_and_mean_sentemb.slurm
sbatch eval_kw_matching_model_vicuna_folder_with_cosdecay_and_mean_sentemb.slurm
sbatch eval_harmbench_model_vicuna_folder_without_cosdecay_and_mean_sentemb.slurm
sbatch eval_llamaguard_model_vicuna_folder_without_cosdecay_and_mean_sentemb.slurm
sbatch eval_kw_matching_model_vicuna_folder_without_cosdecay_and_mean_sentemb.slurm
sbatch eval_harmbench_model_vicuna_folder_without_cosdecay_and_lasthiddenstate.slurm
sbatch eval_llamaguard_model_vicuna_folder_without_cosdecay_and_lasthiddenstate.slurm
sbatch eval_kw_matching_model_vicuna_folder_without_cosdecay_and_lasthiddenstate.slurm
sbat